# Analyse et Visualisation des Données de Laptops

Ce Notebook a pour objectif d'analyser et de visualiser les données relatives aux laptops collectées sur deux plateformes (*eBay* et *Flipkart*).

Les étapes réalisées incluent :

- **Importation des bibliothèques et configuration du logging** : Manipulation des données, visualisation, gestion des chemins, traitement des expressions régulières et journalisation.
- **Configuration des chemins** : Définition des répertoires du projet pour les données nettoyées et les résultats.
- **Chargement des données nettoyées** : Lecture et préparation des fichiers CSV des laptops.
- **Filtrage des produits cross-plateformes** : Identification des produits présents sur les deux plateformes.
- **Analyse et visualisation des différences de prix** : Agrégation des données et génération de graphiques.

Chaque fonction utilisée est expliquée en détail dans la suite du Notebook.


# Importation des Librairies et Configuration du Logging

Dans cette section, nous importons les bibliothèques nécessaires pour :

- La manipulation de données avec **pandas**.
- La création de graphiques avec **matplotlib** et **seaborn**.
- La gestion des chemins avec **pathlib**.
- Le traitement des expressions régulières avec **re**.
- La journalisation des opérations avec **logging**.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import re
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Configuration des Chemins du Projet

Nous définissons ici les chemins d’accès aux différents répertoires du projet :
- `PROJECT_ROOT` : Racine du projet.
- `CLEANED_DATA_PATH` : Dossier contenant les données nettoyées.
- `RESULTS_PATH` : Dossier où seront sauvegardés les résultats (les graphiques générés).


In [ ]:
# Configure paths
PROJECT_ROOT = Path(__file__).resolve().parents[2]
CLEANED_DATA_PATH = PROJECT_ROOT / 'data' / 'cleaned'
RESULTS_PATH = PROJECT_ROOT / 'results'


# Chargement des Données Nettoyées

La fonction `load_cleaned_data` charge les fichiers CSV nettoyés pour la catégorie **laptops** provenant des plateformes *eBay* et *Flipkart*.  
Les étapes réalisées pour chaque fichier sont les suivantes :
- **Standardisation des noms de colonnes** : Conversion en minuscules et remplacement des espaces par des underscores.
- **Validation des colonnes requises** : Vérification de la présence des colonnes nécessaires (*ram*, *cpu*, *brand*, *price*, *collection_date*).
- **Ajout d’une colonne `platform`** : Identification de la plateforme d’origine.
- **Normalisation du CPU et du format de la RAM** :
  - Application de `normalize_cpu_model` sur la colonne `cpu`.
  - Extraction de la valeur numérique pour la RAM.
- **Traitement de la date de collecte** : Conversion de la colonne `collection_date` en date.
  
Les DataFrames ainsi créés sont ensuite concaténés pour obtenir un DataFrame global.


In [ ]:
def load_cleaned_data():
    """Load cleaned data for laptops from eBay and Flipkart"""
    platforms = ['ebay', 'flipkart']
    dfs = []
    
    required_columns = {'ram', 'cpu', 'brand', 'price', 'collection_date'}
    
    for platform in platforms:
        data_dir = CLEANED_DATA_PATH / platform / 'laptops'
        if not data_dir.exists():
            logger.warning(f"Missing data directory: {data_dir}")
            continue
            
        for file in data_dir.glob('*.csv'):
            try:
                df = pd.read_csv(file)
                df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
                
                # Validate columns
                missing_cols = required_columns - set(df.columns)
                if missing_cols:
                    logger.warning(f"Missing columns in {file}: {missing_cols}")
                    continue
                
                # Add platform identifier
                df['platform'] = platform
                
                # Normalize CPU and RAM
                df['cpu_model'] = df['cpu'].apply(normalize_cpu_model)
                df['ram'] = df['ram'].apply(lambda x: int(re.sub(r"\D", "", str(x))) if pd.notnull(x) else None)
                df['brand'] = df['brand'].str.lower().str.strip()
                
                # Handle collection date
                df['collection_date'] = pd.to_datetime(
                    df['collection_date'], errors='coerce'
                ).dt.date
                
                dfs.append(df)
                
            except Exception as e:
                logger.error(f"Error loading {file}: {str(e)}")
                continue
    
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


# Filtrage des Produits Cross-plateformes

La fonction `filter_products_by_platforms` identifie les produits (laptops) disponibles sur les deux plateformes (*eBay* et *Flipkart*).  
Pour ce faire, elle :
- Crée un identifiant unique (`product_id`) pour chaque produit en combinant les informations de **brand**, **cpu_model** et **ram**.
- Regroupe les produits par `product_id` et identifie ceux présents sur les deux plateformes.
  
Le DataFrame retourné ne contient que les produits qui apparaissent sur *eBay* et *Flipkart*.


In [ ]:
def filter_products_by_platforms(df: pd.DataFrame):
    """Find products with matching RAM, CPU, and Brand across platforms"""
    # Create a unique product identifier
    df['product_id'] = df['brand'] + "|" + df['cpu_model'].astype(str) + "|" + df['ram'].astype(str)
    
    # Find products present on both platforms
    platform_groups = df.groupby('product_id')['platform'].unique()
    cross_platform = platform_groups[platform_groups.apply(
        lambda x: len(set(x) & {'ebay', 'flipkart'}) >= 2
    )].index.tolist()
    
    return df[df['product_id'].isin(cross_platform)]


# Analyse et Visualisation des Différences de Prix

La fonction `analyze_price_differences` réalise l’analyse des écarts de prix pour les produits identifiés comme étant présents sur les deux plateformes.  
Les étapes incluent :
- **Agrégation** : Calcul du prix moyen par date de collecte pour chaque produit et par plateforme.
- **Visualisation** : Génération d’un graphique en barres pour chaque produit, affichant l’évolution des prix dans le temps.
- **Sauvegarde des Graphiques** : Chaque graphique est enregistré dans le dossier `RESULTS_PATH / 'laptops'` avec un nom de fichier dérivé de l’identifiant du produit.


In [ ]:
def analyze_price_differences(filtered_df: pd.DataFrame):
    """Analyze and visualize price differences for matched products"""
    if filtered_df.empty:
        logger.info("No cross-platform products found for analysis")
        return
    
    # Group by product_id and platform
    grouped = filtered_df.groupby(['product_id', 'platform', 'collection_date'])['price'].mean().unstack(level='platform')
    
    # Generate bar plots for each product
    for product_id, data in grouped.groupby(level=0):
        plt.figure(figsize=(12, 6))
        
        # Plot bar graph
        data.plot(kind='bar', figsize=(12, 6))
        
        # Set title and labels
        plt.title(f"Price Trends for {product_id.replace('|', ' ')}")
        plt.xlabel("Collection Date")
        plt.ylabel("Price (USD)")
        
        # Format x-axis dates
        plt.xticks(range(len(data.index)), data.index.get_level_values('collection_date'), rotation=45, ha='right')
        
        # Add legend and grid
        plt.legend(title='Platform')
        plt.grid(True)
        plt.tight_layout()
        
        # Save the plot
        sanitized_title = re.sub(r"[^\w\s]", "_", product_id.replace("|", "_"))
        file_name = f"Product_{sanitized_title}.png"
        # Créer le dossier de destination s'il n'existe pas
        (RESULTS_PATH / 'laptops').mkdir(exist_ok=True, parents=True)
        plt.savefig(RESULTS_PATH / 'laptops' / file_name)
        plt.close()


# Exécution Principale

La section suivante orchestre l'exécution complète du processus :

1. **Création des répertoires de résultats** si nécessaire.
2. **Chargement et traitement des données** via `load_cleaned_data()`.
3. **Filtrage des produits cross-plateformes** avec `filter_products_by_platforms()`.
4. **Analyse et visualisation des différences de prix** à l'aide de `analyze_price_differences()`.

En cas d'erreur, un message détaillé est loggé.


In [ ]:
if __name__ == "__main__":
    # Créer les dossiers de résultats s'ils n'existent pas
    RESULTS_PATH.mkdir(exist_ok=True)
    (RESULTS_PATH / 'laptops').mkdir(exist_ok=True)
    
    try:
        logger.info("Loading and processing laptop data...")
        df = load_cleaned_data()
        
        if df.empty:
            logger.error("No cleaned data found. Check data/cleaned directories.")
            exit(1)
            
        logger.info(f"Loaded {len(df)} records from cleaned data")
        
        filtered_df = filter_products_by_platforms(df)
        logger.info(f"Found {len(filtered_df)} cross-platform product entries")
        
        analyze_price_differences(filtered_df)
        
    except Exception as e:
        logger.error(f"Critical error: {str(e)}", exc_info=True)
